In [1]:
// In Deno, we have to prefix the imports from `node_modules` with `npm:`:
import { Ollama } from "npm:@langchain/community/llms/ollama";
import { BufferMemory } from "npm:langchain/memory";
import { LLMChain } from "npm:langchain/chains";
import {
  PromptTemplate,
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "npm:@langchain/core/prompts";
import { StructuredOutputParser } from "npm:langchain/output_parsers";

import * as diff from 'npm:diff';
import { z } from "npm:zod";

import { html } from "https://deno.land/x/display/mod.ts";

#### First step: we send our text to the LLM for review, and we get the model's response as a Javascript object containing the reviewed text

In [27]:
// Basic LangChain chains contain a prompt, a model and an output parser. Our chain:
// 1. creates a prompt from the user inputs: `instruction` and `inputText`
// 2. sends it to an LLM, the LLM responds with a JSON string
// 3. parses the LLM's JOSN output to a Javscript object

// 1. This is our prompt, it has two inputs: `instruction` and `inputText`. We'll assign values for these inputs, when we execute the chain
const prompt = PromptTemplate.fromTemplate(`
{instruction}
{inputText}
`);

// 2. We create an Ollama LangChain LLM model that uses the local codellama:7b-code model. This is a small model, requires about 4 GB of RAM. 
// I use this model on my older laptop that has only 16 GB RAM and a dual core CPU, and it gives responses with an acceptable speed. 
// Bigger models, like llama2:70b give better results, but they require a computer with 64 GB of RAM and a fast CPU/GPU.
const llm = new Ollama({
  baseUrl: "http://localhost:11434",
  model: "codellama:7b-code",
  verbose: false,
});

// 3. Output parser: We define a Zod schema. LangChain will parse the LLM model's output using this schema, and return a Javascript object
//    at the end of the chain in the following data structure: `{ "reviewedText": "the reviewed text" }`
const ResponseZod = z.object({ reviewedText: z.string() });
type ResponseType = z.infer<typeof ResponseZod>;
const outputParser = StructuredOutputParser.fromZodSchema(ResponseZod);

// the chain contains the prompt, the model and the output parser
const chain = prompt.pipe(llm).pipe(outputParser);

// we specify the inputs for the prompt
const instruction = 'Fix the grammar issues in the following text. Return the reviewed text in JSON only, using the following format: ' +
    '{ "reviewedText": "the reviewed text" }';
const inputText = "How to stays relevant as the developer in the world of ai?";

// finally we execute the chain and get the response as a JavaScript object
const response: ResponseType = await chain.invoke({ instruction, inputText });

#### Second step: we compare our original text and the LLM's response, and show the differences

In [28]:
// We compare the original text and the LLM's response
const diffChanges = diff.diffChars(inputText, response.reviewedText);

// We show the differences between our original text and the LLM's response in HTML
const h = diffChanges.map((r) => {
  if (!r.added && !r.removed) { return `<span>${r.value}</span>`; } 
  if (r.added) { return `<span style="color: lightgreen;">${r.value}</span>`; } 
  if (r.removed) { return `<span style="color: red; text-decoration: line-through;">${r.value}</span>`; } 
}).join("");
    
html`<div style="background: black; color: white; padding: 10px;">${h}</div>`;

How to stay s relevant as the a developer in the world of ai AI ?